In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
from optuna import Trial
from typing import Dict, Union, Any
import evaluate

In [2]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_2197/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_2197/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [3]:
train

,label,text,diff
0,1,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
1,1,[CVE-2012-2098] Integrate fallback sort into t...,diff --git a/src/main/java/org/apache/commons/...
2,0,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
3,0,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
4,1,Resolving Issue #918\n\nMinor XSS and create g...,diff --git a/docs/CHANGELOG b/docs/CHANGELOG\n...
5,1,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
6,0,9.0.4,diff --git a/package.json b/package.json\ninde...
7,0,"Needs to allow this syscall as well, caught on...",diff --git a/linux_priv.c b/linux_priv.c\ninde...
8,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
9,1,Update NewslettersController.php\n\nTest if an...,diff --git a/www/guis/users/application/contro...


In [4]:
len(train)

40

In [5]:
train['label'].value_counts()

label
0    25
1    15
Name: count, dtype: int64

In [6]:
test['label'].value_counts()

label
0    6322
1    3750
Name: count, dtype: int64

In [7]:
from sklearn import  metrics

# def compute_metrics(y_pred, y_test):
#     # print(y_pred,y_test)
#     # classification_report = metrics.classification_report(y_test,y_pred)
#     accuracy_score = metrics.accuracy_score(y_test,y_pred)
#     recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
#     f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

#     # return {"classification_report": classification_report}
#     return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [8]:
train

,label,text,diff
0,1,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
1,1,[CVE-2012-2098] Integrate fallback sort into t...,diff --git a/src/main/java/org/apache/commons/...
2,0,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
3,0,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
4,1,Resolving Issue #918\n\nMinor XSS and create g...,diff --git a/docs/CHANGELOG b/docs/CHANGELOG\n...
5,1,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
6,0,9.0.4,diff --git a/package.json b/package.json\ninde...
7,0,"Needs to allow this syscall as well, caught on...",diff --git a/linux_priv.c b/linux_priv.c\ninde...
8,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
9,1,Update NewslettersController.php\n\nTest if an...,diff --git a/www/guis/users/application/contro...


In [9]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "../../bert-base-cased", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained('../../bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [13]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [15]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load('../../evaluate/metrics/accuracy')
    precision =  evaluate.load("../../evaluate/metrics/precision")
    recall =  evaluate.load("../../evaluate/metrics/recall")
    f1 =  evaluate.load("../../evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')

    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [16]:
# !export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000
# !export https_proxy=http://turbo2.gpushare.com:30000 http_proxy=http://turbo2.gpushare.com:30000

In [17]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> model:
    # params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return model
def my_compute_objective(metrics):
    print('+++++++++++',metrics['eval_accuracy']['accuracy'])
    return  metrics['eval_accuracy']['accuracy']

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    model_init= model_init,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:423: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
[I 2024-05-29 15:40:31,626] A new study created in memory with name: no-name-f4899baa-d75c-4a21-87dd-8eef34b4b00d
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_2197/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than to

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.675832,{'precision': 0.6005237898872176},{'recall': 0.5999801429706116},{'f1': 0.6002483802028857},{'accuracy': 0.5999801429706116}


+++++++++++ 0.5999801429706116


[I 2024-05-29 15:41:15,263] Trial 0 finished with value: 0.5999801429706116 and parameters: {'learning_rate': 0.0005419699832078994}. Best is trial 0 with value: 0.5999801429706116.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_2197/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.659184,{'precision': 0.612844748683884},{'recall': 0.6324463860206513},{'f1': 0.6134162618593584},{'accuracy': 0.6324463860206513}


...

+++++++++++ 0.6276806989674345


[I 2024-05-29 15:46:14,930] Trial 8 finished with value: 0.6276806989674345 and parameters: {'learning_rate': 1.2046846744914659e-05}. Best is trial 1 with value: 0.6324463860206513.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_2197/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.933099,{'precision': 0.6566079328797317},{'recall': 0.37817712470214454},{'f1': 0.21717802470706243},{'accuracy': 0.37817712470214454}


+++++++++++ 0.37817712470214454


[I 2024-05-29 15:46:49,379] Trial 9 pruned. 


In [19]:
best_run

BestRun(run_id='1', objective=0.6324463860206513, hyperparameters={'learning_rate': 3.831396789072106e-06}, run_summary=None)